## Generate the plan
Save a game save from which you want to reach a production goal.

In [1]:
START_STATS_FOLDER = 'example/example_data'
# Extract the stats (TODO: set the save file path and uncomment the script)
# START_SAV_FILE = '<your_save_file>.sav'
# !python3 main_game_stats.py {START_SAV_FILE} --print-progress --out {START_STATS_FOLDER}

The goal could be the costs of the milestone or the game phase or something entirely different. Create the suitable problem configuration for it.

In [2]:
# Calculate the plan
RAPID_PROB_CONFIG_FILE = 'example/example_configurations/rapid_production_config.yml'
RAPID_PLAN_OUT_FILE = 'data/rapid_plan_out.yml'

!python3 main_rapid_production.py {RAPID_PROB_CONFIG_FILE} --out {RAPID_PLAN_OUT_FILE}

Number of involved recipes: 98
Search solution in minimal number of steps
-----------------
Test with #steps:  4
Create problem...
Number of variables = 800
Number of constraints = 680
Optimize...
Problem processed in 2358 milliseconds
Iterations sufficient
-----------------
Test with #steps:  2
Create problem...
Number of variables = 400
Number of constraints = 406
Optimize...
Problem processed in 1298 milliseconds
Iterations sufficient
-----------------
Test with #steps:  1
Create problem...
Number of variables = 200
Number of constraints = 269
Optimize...
Problem processed in 317 milliseconds
Iterations insufficient
Minimal number of steps: 2


## Execute the steps in the plan

In [3]:
STEP_ID = 0 # Increase after completing finishing all cells below and repeat
# load libraries
from assistory import game
import yaml
from assistory.optim.rapid_plan import RapidPlan

# Load plan
with open(RAPID_PLAN_OUT_FILE, 'r') as fp:
    plan_data = yaml.safe_load(fp)
plan = RapidPlan.from_dict(plan_data)

recipes_existing = game.RecipeValues.load(START_STATS_FOLDER + '/base_recipe_count.yml')
items_existing = game.ItemValues.load(START_STATS_FOLDER + '/base_items.yml')

In [4]:
# What investment is required?
recipes_automated = plan.steps_recipes_automated[STEP_ID]
buildings_for_step = plan.steps_recipe_factories[STEP_ID].get_buildings()
investment_costs = buildings_for_step.get_costs()

print('Recipes to build:')
recipes_automated.round(3).pprint(0)
print('\nBuildings to build:')
buildings_for_step.round(3).pprint(0)
print('\nInvestment costs:')
investment_costs.round(3).pprint(0)
# Collect investment costs

Recipes to build:
Recipe_Alternate_ReinforcedIronPlate_2_C 2.0
Recipe_EncasedIndustrialBeam_C           1.0
Recipe_Stator_C                          1.0

Buildings to build:
Desc_AssemblerMk1_C 4.0

Investment costs:
Desc_Cable_C               40.0
Desc_IronPlateReinforced_C 32.0
Desc_Rotor_C               16.0


Compare existing items with target

In [5]:
CURRENT_SAV_FILE = 'data/Assistory-phase-3.sav' # TODO: set to current save file path
CURRENT_STATS_OUT_FOLDER = 'data/stats/phase-3'
# iteratively save the game state to CURRENT_SAV_FILE and rerun the cell
!python3 main_game_stats.py {CURRENT_SAV_FILE} --out {CURRENT_STATS_OUT_FOLDER}
items_current = game.ItemValues.load(CURRENT_STATS_OUT_FOLDER + '/base_items.yml')

print('Investment costs reached? (Existing/Target)')
for item_name in investment_costs.as_dict_ignoring(0):
    current_amount = items_current[item_name]
    target_amount = investment_costs[item_name]
    reached = 'v' if current_amount >= target_amount else 'x'
    print(f'{reached} {item_name}: {current_amount}/{target_amount}')

Investment costs reached? (Existing/Target)
v Desc_IronPlateReinforced_C: 148/32.0
v Desc_Rotor_C: 78/16.0
v Desc_Cable_C: 1036/40.0


Now, build the factories required to execute the automated production in the steps. Hint: Only the input items for the step duration must be provided to the factories.

In [6]:
# iteratively save the game state to CURRENT_SAV_FILE and rerun the cell
!python3 main_game_stats.py {CURRENT_SAV_FILE} --out {CURRENT_STATS_OUT_FOLDER}
recipes_current = game.RecipeValues.load(CURRENT_STATS_OUT_FOLDER + '/base_recipe_count_rounded.yml')

recipes_target_rounded = (recipes_automated + recipes_existing).round(3)
recipes_existing_rounded = recipes_existing.round(3)
print('\nRecipes reached? (Existing/Target)')
for recipe_name in recipes_automated.round(3).as_dict_ignoring(0):
    current_amount = recipes_current[recipe_name]
    target_amount = recipes_target_rounded[recipe_name]
    reached = 'v' if current_amount >= target_amount else 'x'
    print(f'{reached} {recipe_name}: {current_amount}/{target_amount}')
# Build the factories with required recipes at the correct clock rate


Recipes reached? (Existing/Target)
x Recipe_Alternate_ReinforcedIronPlate_2_C: 3.2/5.2
x Recipe_EncasedIndustrialBeam_C: 0/1.0
x Recipe_Stator_C: 0.2/1.2


What manual work needs to be done during step?

In [7]:
import main_rapid_production
rapid_conf = main_rapid_production.RapidProductionProblemUserConfig.load_from_file(
    RAPID_PROB_CONFIG_FILE
).get_problem_configuration()

recipes_handcraft = plan.steps_recipes_handcraft[STEP_ID]
recipes_handcraft_step = recipes_handcraft * rapid_conf.step_duration

print('Time of execute handcraft recipes (in seconds):')
handcraft_duration = (rapid_conf.step_duration * 60) * recipes_handcraft
handcraft_duration.round(3).pprint(ignore_value=0)
print('\nHandcraft production goal:')
recipes_handcraft_step.get_item_rate_balance_handcraft().round(3).pprint(ignore_value=0)
# Build the required items during the step duration

Time of execute handcraft recipes (in seconds):
Recipe_EncasedIndustrialBeam_C 61.667
Recipe_HandcraftStone_C        3.458
Recipe_IronPlateReinforced_C   5.625
Recipe_ModularFrame_C          93.75
Recipe_PortableMiner_C         30.0

Handcraft production goal:
BP_ItemDescriptorPortableMiner_C 6.0
Desc_Cement_C                    -296.0
Desc_IronPlateReinforced_C       -71.25
Desc_IronPlate_C                 -34.5
Desc_IronRod_C                   -324.0
Desc_IronScrew_C                 -45.0
Desc_ModularFrame_C              50.0
Desc_SteelPlateReinforced_C      49.333
Desc_SteelPlate_C                -148.0
Desc_Stone_C                     4.611


Is step finalized? This is the case if the item amount is reached

In [8]:
# iteratively save the game state to CURRENT_SAV_FILE and rerun the cell
!python3 main_game_stats.py {CURRENT_SAV_FILE} --out {CURRENT_STATS_OUT_FOLDER}
items_current = game.ItemValues.load(CURRENT_STATS_OUT_FOLDER + '/base_items.yml')

items_step_target = items_existing
for step_i in range(STEP_ID + 1):
    step_item_rate = plan.get_item_rates(step_i)
    items_step_target += step_item_rate * rapid_conf.step_duration

print('\nStep target reached? (Existing/Target)')
items_step_target = items_step_target.round(4)
for item_name in sorted(items_step_target.as_dict_ignoring(0)):
    current_amount = items_current[item_name]
    target_amount = items_step_target[item_name]
    if current_amount == target_amount:
        continue
    reached = 'v' if current_amount >= target_amount else 'x'
    print(f'{reached} {item_name}: {current_amount}/{target_amount}')
# Produce until reached


Step target reached? (Existing/Target)
x BP_ItemDescriptorPortableMiner_C: 1/6.0
v Desc_Cable_C: 1036/946.0
v Desc_Cement_C: 500/120.0
v Desc_IronPlateReinforced_C: 148/44.0
v Desc_IronPlate_C: 474/254.0
v Desc_IronRod_C: 377/57.0
v Desc_IronScrew_C: 344/299.0
x Desc_ModularFrame_C: 4/50.0
x Desc_Rotor_C: 78/92.0
x Desc_Stator_C: 1/25.0
v Desc_SteelPipe_C: 212/46.0
v Desc_SteelPlateReinforced_C: 96/79.3333
v Desc_SteelPlate_C: 200/153.0
x Desc_Stone_C: 0/4.6111
v Desc_Wire_C: 934/613.0


Disassemble the added factories

In [9]:
# Is step finalized? This is the case if the item amount is reached
!python3 main_game_stats.py {CURRENT_SAV_FILE} --out {CURRENT_STATS_OUT_FOLDER}
recipes_current = game.RecipeValues.load(CURRENT_STATS_OUT_FOLDER + '/base_recipe_count.yml')

recipes_current_rounded = recipes_current.round(3)
recipes_existing_rounded = recipes_existing.round(3)
print('\nRecipes disassembled? (Existing/Target)')
for recipe_name in sorted(recipes_automated.as_dict_ignoring(0)):
    current_amount = recipes_current_rounded[recipe_name]
    target_amount = recipes_existing_rounded[recipe_name]
    reached = 'v' if current_amount == target_amount else 'x'
    print(f'{reached} {recipe_name}: {current_amount}/{target_amount}')


Recipes disassembled? (Existing/Target)
v Recipe_Alternate_ReinforcedIronPlate_2_C: 3.2/3.2
v Recipe_EncasedIndustrialBeam_C: 0/0
v Recipe_Stator_C: 0.2/0.2
